In [18]:
import pandas as pd

In [19]:
# 빅카인즈에서 내려받은 엑셀 업로드
df = pd.read_csv('./성범죄_0826_0902.csv')

In [20]:
# 제목만 따오기
title_list=df['제목']
title_list

0      ‘미성년 11명 성폭행’ 김근식 내달 출소 법무부 “1대 1 전자감독 대상”
1                     [단독] 교사도, 강사도 ‘엘’은 어디에나 있었다
2                 미성년 12명 성폭행 김근식, 10월 출소 불안한 주민들
3                  "야채만 먹였다" 18개월 아들 영양실조로 죽게한 엄마
4                 미성년 11명 유린 성폭행범 출소 "전자발찌 채운다는데"
                          ...                    
470          “명백한 동의 없다면 성폭행” 스페인 ‘비동의 강간죄’ 만들어진다
471                                     [분수대] 음성권
472         미성년자 성매매 강요 영리 취득 검찰, 조직 1명에 징역 7년 구형
473            성추행 갑질논란 체육회 전간부, 벌금형 40시간 치료교육 명령
474              [꿈꾸는 경기교육] 낙태가 아닌 임신 중단으로 불리는 그날
Name: 제목, Length: 475, dtype: object

# 네이버 댓글 url 가져오기

In [21]:
from selenium import webdriver
import time

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)

In [22]:
# 매크로 방지 피하기 위해 wait을 걸어줍니다
driver.implicitly_wait(2)

# url 넣을 리스트 생성
url_list = []

for title in title_list:
    try:
        url_format = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_dgs&qdt=0'
        # 검색 결과 창 띄우기
        driver.get(url_format.format(title))
        time.sleep(2)

        # 기사 섹션 접근
        list_news = driver.find_element_by_class_name('group_news')
        li = list_news.find_element_by_tag_name('li')

        # '네이버뉴스' 클릭할 수 있는 상단 섹션 접근
        info_group = li.find_element_by_class_name('info_group')

        # 모든 url 가져오기
        a_s = info_group.find_elements_by_tag_name('a')

        # url 중 네이버뉴스에 해당하는 url만 리스트에 추가
        for a in a_s:
            url = a.get_attribute('href')

            if 'naver' in url:
                url_list.append(url)
    except:
        continue

In [23]:
# 스포츠뉴스 url은 형식이 달라서 삭제 처리    
url_list = [s for s in url_list if "sports" not in s] 

In [49]:
len(url_list)

430

# 댓글 url에서 댓글 크롤링

In [47]:
driver = webdriver.Chrome('./chromedriver.exe')
comment_df = pd.DataFrame(columns={'comment','title'})

# 클린봇 버튼을 1번만 눌러야해서 1번째 url만 실행
for url in url_list[:1]:
    idx = url.find('article')
    url_new = url[:idx+7]+'/comment'+url[idx+7:]

    #웹 드라이버
    driver.implicitly_wait(3)
    driver.get(url_new)

    #네이버의 경우, 클린봇으로 추출이 안되는게 있다, 클린봇 옵션 해제 후 추출해주도록 한다.
    cleanbot = driver.find_element_by_css_selector('a.u_cbox_cleanbot_setbutton') # 클린봇 설정 버튼
    cleanbot.click()
    time.sleep(1)
    cleanbot_disable = driver.find_element_by_xpath("//*[@id='cleanbot_dialog_checkbox_cbox_module']") # 클린봇 해제 버튼
    cleanbot_disable.click()
    time.sleep(1)
    cleanbot_confirm = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]/div[4]/button") # 확인 버튼
    cleanbot_confirm.click()
    time.sleep(1)

    #더보기 계속 클릭하기
    cnt = 0
    while True:
        try:
            btn_more = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            btn_more.click()
            # time.sleep(1)
        except:
            break
    
    #제목추출
    title = driver.find_element_by_class_name('media_end_head_headline')
    #댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')

    for content in contents:
        comment_df = comment_df.append({'comment':content.text, 'title':title.text}, ignore_index=True)

In [51]:
# 클린봇 버튼 누르지않고 실행
for url in url_list[1:]:

    idx = url.find('article')
    url_new = url[:idx+7]+'/comment'+url[idx+7:]

    #웹 드라이버
    driver.implicitly_wait(3)
    driver.get(url_new)


    #더보기 계속 클릭하기
    cnt = 0
    while True:
        try:
            btn_more = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            btn_more.click()
            # time.sleep(1)
        except:
            break

    #제목추출
    title = driver.find_element_by_class_name('media_end_head_headline')
    #댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')

    for content in contents:
        comment_df = comment_df.append({'comment':content.text, 'title':title.text}, ignore_index=True)

KeyboardInterrupt: 

In [57]:
len(comment_df.title.unique())

215

In [62]:
comment_df['title'].value_counts().head(20)

초중고 11명 성폭행…조두순 뺨치는 김근식 다음달 나온다                  1492
조두순보다 더한 그가 출소한다 '미성년자 11명 연쇄 성폭행'               1320
"출소 16일만에 9살 성폭행"…미성년자 11명 연쇄 성폭행범 21일 나온다       1277
아빠·감독이 16살 女선수 ‘궁디팡팡’… 성추행 논란                    1086
초중고 11명 성폭행... 조두순 뺨치는 김근식, 10월 사회 나온다            849
교사 수업 중인데 교단에 누워 휴대전화 '만지작'…"교권 추락의 민낯"           822
"다른 남성이 엉덩이 만지던데" 이태원에 나타난 비키니 오토바이女 "딱히.."       811
[단독]교사 “식판 맞아도 참는다”… 교권침해 3년간 6128건, 고발 14건뿐      709
‘성폭력 폭로’ 강민진 “당기위서 팬티만 입었는지 물어”...2차 가해 문제 제기     693
미성년 11명 성폭행, 조두순만큼 끔찍한 그놈 나온다                     689
16살 딸 '궁디팡팡'은 성추행일까…국제 테니스 대회서 때아닌 논란             680
무너지는 교권...수업 중 교단에 벌러덩 누운 학생, 그대로 수업한 교사          657
여중생에 “섹시해” “너 없이 못 살아”… 학교는 남교사 감쌌다               597
"도와달라"며 미성년 11명 성폭행…'조두순급 악질' 김근식 곧 출소            562
출소 16일만 미성년 11명 성폭행…조두순 뺨친 흉악범이 출소한다              556
“아가씨라고 부르지 마세요” 호칭 갈등 어떻게 할까요?                    493
'미성년 12명 성폭행' 김근식 10월 출소…맘카페 '발칵'                 459
‘성 접대 의혹’ 이준석 전 대표 ‘경찰 소환 조사’                     455
“도와달라”며 유인해 11명 성폭행…김근식 내달 출

In [74]:
comment_df

,comment,title
0,11명 성폭행하고 15년이라는게 더 충격이다ㅋㅋ대한민국 형 부여 어이상실,‘미성년 11명 성폭행’ 김근식 내달 출소… 법무부 “1대 1 전자감독 대상”
1,사형좀 시키지 15년동안 밥을 먹이다니 ...,‘미성년 11명 성폭행’ 김근식 내달 출소… 법무부 “1대 1 전자감독 대상”
2,"영미권(미국, 영국, 캐나다, 호주, 뉴질랜드 포함) 국가였으면 진짜 길가다가 총맞...",‘미성년 11명 성폭행’ 김근식 내달 출소… 법무부 “1대 1 전자감독 대상”
3,김근식 출소해서 사고치면 누구 하나 책임지라니까,‘미성년 11명 성폭행’ 김근식 내달 출소… 법무부 “1대 1 전자감독 대상”
4,초등학생???? 11명???? 15년????!!!! 거기다 1대1감시??이거 뭐하는...,‘미성년 11명 성폭행’ 김근식 내달 출소… 법무부 “1대 1 전자감독 대상”
...,...,...
27375,요즘사람이면 이해하지만 옛날분들한테 그런식으로 말하면 안되지 니들세대 이해해 달라고...,알바생을 ‘아가씨’라 불렀다가 욕 먹었네요 [사연뉴스]
27376,나는 무조건 사장님 이라고 부른다. 그게 편하더라. 사장님~ 부르면 사장이 오거나 ...,알바생을 ‘아가씨’라 불렀다가 욕 먹었네요 [사연뉴스]
27377,예민한것도 맞고ㅋㅋ 근데 남알바생한테 총각이라고 하는지 생각해보면 답나올껄 총각이란...,알바생을 ‘아가씨’라 불렀다가 욕 먹었네요 [사연뉴스]
27378,숙녀분이라고 불렀어야죠...ㅋㅋㅋ..숙녀분....여기 돼지갈비 2인분 추가요.,알바생을 ‘아가씨’라 불렀다가 욕 먹었네요 [사연뉴스]


In [75]:
# 댓글 27380개 엑셀 저장
comment_df.to_excel('./0927_성범죄_댓글_1.xlsx',index=False)